In [1]:
import pandas as pd
import numpy as np
import matplotlib as py
import csv

In [2]:
# read the air pollution file
air_death_df = pd.read_csv('../resources/7.8.20 UCSD Group Project Dataset - death-rates-from-air-pollution.csv')
# read the happiness files
happiness_2015_raw_df = pd.read_csv('../resources/2015_happiness_data.csv')
happiness_2016_raw_df = pd.read_csv('../resources/2016_happiness_data.csv')
happiness_2017_raw_df = pd.read_csv('../resources/2017_happiness_data.csv')

In [13]:
air_death_df = air_death_df.rename(columns={'Entity': 'Country'})
air_death_df

,Country,Code,Year,"Air pollution (total) (deaths per 100,000)","Indoor air pollution (deaths per 100,000)","Outdoor particulate matter (deaths per 100,000)","Outdoor ozone pollution (deaths per 100,000)"
0,Afghanistan,AFG,1990,299.477309,250.362910,46.446589,5.616442
1,Afghanistan,AFG,1991,291.277967,242.575125,46.033841,5.603960
2,Afghanistan,AFG,1992,278.963056,232.043878,44.243766,5.611822
3,Afghanistan,AFG,1993,278.790815,231.648134,44.440148,5.655266
4,Afghanistan,AFG,1994,287.162923,238.837177,45.594328,5.718922
...,...,...,...,...,...,...,...
6463,Zimbabwe,ZWE,2013,143.850145,113.456097,27.589603,4.426291
6464,Zimbabwe,ZWE,2014,138.200536,108.703566,26.760618,4.296971
6465,Zimbabwe,ZWE,2015,132.752553,104.340506,25.715415,4.200907
6466,Zimbabwe,ZWE,2016,128.692138,100.392287,25.643570,4.117173


In [14]:

# air_year = air_death_df.groupby(['Year'])
# air_year.mean()
# air_year.mean().shape
# test = air_year.drop(["1990"])

In [26]:
# Remove data from years 1990-2014
# first set the index by year
index_df = air_death_df.set_index('Year')
index_df

,Country,Code,"Air pollution (total) (deaths per 100,000)","Indoor air pollution (deaths per 100,000)","Outdoor particulate matter (deaths per 100,000)","Outdoor ozone pollution (deaths per 100,000)"
Year,,,,,,
1990,Afghanistan,AFG,299.477309,250.362910,46.446589,5.616442
1991,Afghanistan,AFG,291.277967,242.575125,46.033841,5.603960
1992,Afghanistan,AFG,278.963056,232.043878,44.243766,5.611822
1993,Afghanistan,AFG,278.790815,231.648134,44.440148,5.655266
1994,Afghanistan,AFG,287.162923,238.837177,45.594328,5.718922
...,...,...,...,...,...,...
2013,Zimbabwe,ZWE,143.850145,113.456097,27.589603,4.426291
2014,Zimbabwe,ZWE,138.200536,108.703566,26.760618,4.296971
2015,Zimbabwe,ZWE,132.752553,104.340506,25.715415,4.200907


In [46]:
# Remove data from years 1990-2014
# use .drop() and set the range(1990, 2015)
clean_year_df = index_df.drop(range(1990, 2015), axis=0)
clean_year_df.shape # check the shape to see if rows were deleted

(693, 6)

In [48]:
clean_year_df.loc(['2015'],:)

SyntaxError: invalid syntax (<ipython-input-48-c27340613001>, line 1)